In [28]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.examples.tutorials.mnist import input_data
# 1 load data
# load data  2 one_hot : 1 0000  1 filename
# 2 knn test train distance
# 3 knn k个最近的图片  500 1-》500 train 4
# 4 k个最近的图片  -> 解析
# 5 最后 获取具体的数字
# 6 检测概率统计
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)
#属性设置
trainNum=55000
testNum=10000
trainSize=55000
testSize=10000
k=4
#数据分解  trainSize 个  返回 0-trainNum 之间  replace 不可重复
trainIndex = np.random.choice(trainNum,trainSize,replace=False)
testIndex = np.random.choice(testNum,testSize,replace=False)
trainData = mnist.train.images[trainIndex] #获取训练图片
trainLabel = mnist.train.labels[trainIndex] #获取训练标签
testData = mnist.test.images[testIndex] #获取测试图片
testLabel = mnist.test.labels[testIndex] #获取测试标签

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [29]:
print(trainData.shape)
print(trainLabel.shape)
print(testData.shape)
print(testLabel.shape)

(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)


In [41]:
num=10
# 2 knn距离计算
trainDataInput = tf.placeholder(shape=[None,784],dtype=tf.float32)
trainLabelInput = tf.placeholder(shape=[None,10],dtype=tf.float32)
testDataInput = tf.placeholder(shape=[None,784],dtype=tf.float32)
testLabelInput = tf.placeholder(shape=[None,10],dtype=tf.float32)
# knn distance 5*784  5*1*784
# 5 500  784  （3维）  2500*784
f1=tf.expand_dims(testDataInput,1) #维度扩展
f2=tf.subtract(trainDataInput,f1) # 784 sum(784)
f3=tf.reduce_sum(tf.abs(f2),reduction_indices=2) #完成数据累加  784
# 5*500

f4 = tf.negative(f3)
f5,f6=tf.nn.top_k(f4,k=4) # 选取f4 最大的四个值
#f3中最小的四个值

#f6是下标  index ->label
f7 = tf.gather(trainLabelInput,f6)

f8 = tf.reduce_sum(f7,reduction_indices=1)
f9=tf.argmax(f8,axis=1)

with tf.Session() as sess:
    # f1<-testData 5张图片
    p1 = sess.run(f1,feed_dict={testDataInput:testData[0:num]})
    print('p1=',p1.shape) # p1= (5, 1, 784)
    p2 = sess.run(f2,feed_dict={trainDataInput:trainData,testDataInput:testData[0:num]})
    print('p2=',p2.shape) # p2= (5, 500, 784)
    p3 = sess.run(f3,feed_dict={trainDataInput:trainData,testDataInput:testData[0:num]})
    print('p3=',p3.shape) # p3= (5, 500) 最核心的距离的计算
    print('p3[0,0]=',p3[0,0])
    p4=sess.run(f4,feed_dict={trainDataInput:trainData,testDataInput:testData[0:num]})
    print('p4=',p4.shape) # p3= (5, 500) 最核心的距离的计算
    print('p4[0,0]=',p4[0,0])
    # p6下标 p5是值  5张图的 距离最小的4个图
    p5,p6=sess.run((f5,f6),feed_dict={trainDataInput:trainData,testDataInput:testData[0:num]})
    print('p5=',p5.shape,'p6=',p6.shape)
    print('p5[0,0]=',p5[0,0],'p6[0,0]=',p6[0,0])
    p7 = sess.run(f7,feed_dict={trainDataInput:trainData,testDataInput:testData[0:num],trainLabelInput:trainLabel})
    print('p7=',p7.shape)
#     print(p7)
    p8 = sess.run(f8,feed_dict={trainDataInput:trainData,testDataInput:testData[0:num],trainLabelInput:trainLabel})
    print('p8=',p8.shape)
#     print(p8)
    p9 = sess.run(f9,feed_dict={trainDataInput:trainData,testDataInput:testData[0:num],trainLabelInput:trainLabel})
    print('p9=',p9.shape)
    print('p9=',p9)
    p10 = np.argmax(testLabel[0:num],axis=1)
    print('p10=',p10)
j=0
for i in range(0,num):
    if p10[i]==p9[i]:
        j=j+1
print('ac=',j*100/num)

p1= (10, 1, 784)
p2= (10, 55000, 784)
p3= (10, 55000)
p3[0,0]= 180.85492
p4= (10, 55000)
p4[0,0]= -180.85492
p5= (10, 4) p6= (10, 4)
p5[0,0]= -95.08627 p6[0,0]= 18758
p7= (10, 4, 10)
p8= (10, 10)
p9= (10,)
p9= [8 7 1 1 5 0 5 2 9 8]
p10= [8 7 1 1 5 0 5 2 9 8]
ac= 100.0
